In [4]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import datetime as datetime
import requests
import time

load_dotenv()

LASTFM_KEY = os.getenv('LASTFM_KEY')
LASTFM_SECRET = os.getenv('LASTFM_SECRET')

def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': 'agrant117'}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = LASTFM_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

# r = lastfm_get({'method': 'chart.gettopartists'})
# r.status_code

def jsonPrint(obj):
    # create a formatted string of the Python JSON object
    result = json.dumps(obj, sort_keys=True, indent=4)
    print(result)

# jsonPrint(r.json())

def lookupArtist(artist):
    response = lastfm_get({
        'method': 'artist.getTopTracks',
        'artist':  artist
    })

    # if there's an error, just return nothing
    if response.status_code != 200:
        return None

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return response

# r = lookupArtist('Lupe Fiasco')
# jsonPrint(r.json())


def lookupTrack(artist, track, dict):
    payload = {
        'method': 'track.getInfo',
        'track':  track,
        'artist':  artist
    }
    response = lastfm_get(payload)

    # if there's an error, just return nothing
    if response.status_code != 200:
        return None

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    
    lookupResults = response.json()
    # print(lookupResults)

    if 'track' not in lookupResults:
        dict['artist_mbid'].append('N/A')
        dict['artist_name'].append(artist)
        dict['track_name'].append(track)
        dict['listeners'].append(0)
        dict['playcount'].append(0)
    else:
        if 'mbid' not in lookupResults['track']['artist']:
            dict['artist_mbid'].append('N/A')
        else:
            dict['artist_mbid'].append(lookupResults['track']['artist']['mbid'])

        dict['artist_name'].append(lookupResults['track']['artist']['name'])
        dict['track_name'].append(lookupResults['track']['name'])
        dict['listeners'].append(lookupResults['track']['listeners'])
        dict['playcount'].append(lookupResults['track']['playcount'])


HHPlaylistDF = pd.read_excel('../Spotify/SpotifyPlaylistData.xlsx',sheet_name='HHPlaylist')
CHHPlaylistDF = pd.read_excel('../Spotify/SpotifyPlaylistData.xlsx',sheet_name='CHHPlaylist')

def createTracksDict(dataFrame):
    trackDict = {'artist_mbid': [], 'artist_name': [], 'track_name': [], 'listeners': [], 'playcount': []}

    for idx, item in dataFrame.iterrows():
        # print(item['artist_name'] + ' - ' + item['track_name'])
        lookupTrack(item['artist_name'], item['track_name'], trackDict)
        time.sleep(0.25)

    return trackDict

def writeFile(trackDict, fileName):
    with open(fileName, 'w') as outfile:
        json.dump(trackDict, outfile)

writeFile(createTracksDict(HHPlaylistDF), 'HHTracksDict.json')
writeFile(createTracksDict(CHHPlaylistDF), 'CHHTracksDict.json')

Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 8


In [5]:
def openFileToDataFrame(fileName):
    with open(fileName, 'r') as openfile:
        file = json.load(openfile)

    df = pd.DataFrame(file)
    return df

xlxWriter = pd.ExcelWriter('LastFMData.xlsx', engine='xlsxwriter')

openFileToDataFrame('HHTracksDict.json').to_excel(xlxWriter, sheet_name='HHLastFM')
openFileToDataFrame('CHHTracksDict.json').to_excel(xlxWriter, sheet_name='CHHLastFM')

xlxWriter.close()